In [1]:
import os
import sys
import numpy as np
import torch
from torch import nn
from transformers import pytorch_utils as torch_utils
from peft import LoraConfig

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [3]:
import importlib
import src.train
import src.model

importlib.reload(src.train)
importlib.reload(src.model)

from src.train import sft_train_lora
from src.model import identify_target_modules
from data.logiQA import LogiQA

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [5]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import login
login()

In [ ]:
# dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

dataset = LogiQA(tokenizer=tokenizer)

In [8]:
dataset[50]

{'input_ids': tensor([    2,   250, 16010,    50,  1015,  1059, 31703,    11,     5,   709,
             9,   750,     6,    53,    63,  2788,    40,    28,  1595,   159,
             4,    22,  8138, 11313,   113,    16,   215,    10,  2788,     4,
         34421, 10974,   303,    42,  2788, 18109,     4,   572, 21252,    22,
          8138, 11313,  1297,    89,    16,   117,  2788,    13,    22, 16466,
          1297,    53,    89,    32, 14301,    13,    22, 31421,  1297,    22,
            29,  8310,  1297,     8,    22, 36238,   845,  9068,     6,  2320,
         20947,    14,     5, 17116,    50,  4694,    14,   304,     5,  2136,
            22,  8138, 11313,   113,    33,  9644,  3033,    11,    10,  2569,
           317,   409,    31,     5,  6444,     4,  6834,     9,     5,   511,
             6,   114,  1528,     6,    74,   144, 15310,     5,  6116,     9,
             5,  1065,  2320,   116,   345,    16,   117,  2136,    13,    22,
         26601,   113,    11,    22,  8

In [12]:
len(dataset)

7376

In [9]:
target_modules = identify_target_modules(model, name_segment='self_attn')
print(target_modules)

In [10]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

In [11]:
sft_train_lora(
    base_model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=AutoTokenizer.from_pretrained("facebook/opt-350m"),
    adapter_name="sft_lora",
    response_template=" ### Answer:",
    lora_config=lora_config,
)

/opt/anaconda3/envs/reasoning-decomp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/2766 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 9.05 GB, other allocations: 14.69 MB, max allowed: 9.07 GB). Tried to allocate 10.91 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).